# Step-3 Notebook for combining the Flowcam and IFCB datasets and storing the training, validation and test datasets

In [1]:
import os, glob
import numpy as np
import pandas as pd

In [2]:
import os
import glob
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import sys
from os import listdir, path
from pathlib import Path
from typing import List, Tuple
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
import pickle

### Load the stage 1 processed data for each of flowcam and ifcb datasets

In [3]:
fc_data = os.listdir("./flowcam_processed_1/")

In [4]:
ifcb_data = os.listdir("./ifcb_processed_1/")

### Prepare a common category list between two datasets

In [5]:
common_list=[]
for i in fc_data:
    if i == ".ipynb_checkpoints":
        continue
    if i in ifcb_data:
        common_list.append(i)

In [6]:
common_list.__len__()

0

In [7]:
j=0
common_cats_path_ifcb=[]
for i in glob.glob("./ifcb_processed_1/*"):
    idx = os.path.basename(i)
    if idx in common_list:
        j+=1
        print(j,'----',i)
        common_cats_path_ifcb.append(i)
        
j=0
common_cats_path_fc=[]
for i in glob.glob("./flowcam_processed_1/*"):
    idx = os.path.basename(i)
    if idx in common_list:
        j+=1
        print(j,'----',i)
        common_cats_path_fc.append(i)

In [8]:
common_cats_path_fc.__len__(),common_cats_path_ifcb.__len__()

(0, 0)

In [9]:
k=0
for i in common_cats_path_fc:
    k+=1
    for j in glob.glob(f'{i}/*'):
        if j.split('.')[-1] == 'npy':
            print(k,'--',os.path.basename(i),'--',os.path.basename(j))
            data = np.load(j,allow_pickle=True)
            print(data.shape)

In [10]:
k=0
for i in common_cats_path_ifcb:
    k+=1
    for j in glob.glob(f'{i}/*'):
        if j.split('.')[-1] == 'npy':
            print(k,'--',os.path.basename(i),'--',os.path.basename(j))
            data = np.load(j,allow_pickle=True)
            print(data.shape)

### Add and merge the dataset from Flowcam data to combined processed stage-1 dataset

In [11]:
fc_path = glob.glob("./flowcam_processed_1/*")
common_dir="./common_processed_data"
os.makedirs(common_dir, exist_ok= True)
k=0
not_included=0
common=0
unique=0
for i in fc_path:
    k+=1
    for j in glob.glob(f'{i}/*'):
        if j.split('.')[-1] == 'npy':
            data = np.load(j,allow_pickle=True)
            if data.shape[0] == 1:
                print(f"{k} not included",'--',os.path.basename(i),'--',os.path.basename(j))
                print(data.shape)
                not_included+=1
            elif i in common_cats_path_fc:
                print(f"{k} belongs to common cat",'--',os.path.basename(i),'--',os.path.basename(j))
                for l in common_cats_path_ifcb:
                    if os.path.basename(l) == os.path.basename(i):
                        common+=1
                        dir_path= os.path.join(common_dir,os.path.basename(i))
                        print(os.path.basename(l),os.path.basename(i))
                        data1 = np.load(j,allow_pickle=True)
                        for m in glob.glob(f'{l}/*'):
                            if m.split('.')[-1] == 'npy':
                                data2 = np.load(m,allow_pickle=True)
                        data3=np.empty(data1.shape[0]+data2.shape[0],dtype=object)
                        data3[:data1.shape[0]]=data1
                        data3[data1.shape[0]:]=data2
                        print(data1.shape,data2.shape,data3.shape,data3[:data1.shape[0]][-1].shape,data3[data1.shape[0]:][-1].shape)
                        print(data1[-1].shape,data2[-1].shape)
                        os.makedirs(dir_path,exist_ok=True)
                        path_save = os.path.join(dir_path,os.path.basename(j))
                        np.save(path_save,data3)
                        print("unique saved at--> ",path_save)
            else:
                dir_path= os.path.join(common_dir,os.path.basename(i))
                print(f"{k} passed creating {dir_path}")
                os.makedirs(dir_path,exist_ok=True)
                path_save = os.path.join(dir_path,os.path.basename(j))
                np.save(path_save,data)
                print("unique saved at--> ",path_save)
                unique+=1
print('not_included-->',not_included,'common-->',common,'unique-->',unique)

1 passed creating ./common_processed_data/Euterpina_copepod
unique saved at-->  ./common_processed_data/Euterpina_copepod/Euterpina_copepod_im.npy
2 passed creating ./common_processed_data/Empty_lorica
unique saved at-->  ./common_processed_data/Empty_lorica/Empty_lorica_im.npy
3 passed creating ./common_processed_data/Echinoderm_larvae
unique saved at-->  ./common_processed_data/Echinoderm_larvae/Echinoderm_larvae_im.npy
4 passed creating ./common_processed_data/Eutintinnus_tintinnid
unique saved at-->  ./common_processed_data/Eutintinnus_tintinnid/Eutintinnus_tintinnid_im.npy
not_included--> 0 common--> 0 unique--> 4


In [12]:
os.listdir("./common_processed_data").__len__()

4

### Add and merge the datasets from IFCB data to combined processed stage-1 dataset

In [13]:
fc_path = glob.glob("./flowcam_processed_1/*")
ifcb_path = glob.glob("./ifcb_processed_1/*")
common_dir="./common_processed_data"
k=0
not_included=0
common=0
unique=0
for i in ifcb_path:
    k+=1
    for j in glob.glob(f'{i}/*'):
        if j.split('.')[-1] == 'npy':
            data = np.load(j,allow_pickle=True)
            if data.shape[0] == 1:
                print(f"{k} not included",'--',os.path.basename(i),'--',os.path.basename(j))
                print(data.shape)
                not_included+=1
            elif i in common_cats_path_ifcb:
                print(f"{k} belongs to common cat",'--',os.path.basename(i),'--',os.path.basename(j))
                common+=1
            else:
                dir_path= os.path.join(common_dir,os.path.basename(i))
                print(f"{k} passed creatiing {dir_path}")
                os.makedirs(dir_path,exist_ok=True)
                path_save=os.path.join(dir_path,os.path.basename(j))
                np.save(path_save,data)
                unique+=1
print('not_included-->',not_included,'common-->',common,'unique-->',unique)

1 passed creatiing ./common_processed_data/Acantharian
2 passed creatiing ./common_processed_data/Alexandrium
3 passed creatiing ./common_processed_data/Acanthoica_quattrospina
4 passed creatiing ./common_processed_data/Amphorelloides_tropidoneis
5 passed creatiing ./common_processed_data/Amphidinium_crassipes
6 passed creatiing ./common_processed_data/Askenasia
7 passed creatiing ./common_processed_data/Asterompalus_flabellatus
not_included--> 0 common--> 0 unique--> 7


In [14]:
np.unique(os.listdir("./common_processed_data")).__len__()

11

In [15]:
k=0
for m,i in enumerate(glob.glob("./common_processed_data/*")):
    for j in glob.glob(f"{i}/*.npy"):
        data=np.load(j, allow_pickle=True)
        n=len(data)
        df=pd.DataFrame(columns=['_target'])
        df._target=np.ones(n)*m
        df.to_csv(os.path.join(i,f"{os.path.basename(i)}_pd.csv"))
        print(m,'----',df._target[0],'----',df.shape,data.shape)
        

0 ---- 0.0 ---- (2, 1) (2,)
1 ---- 1.0 ---- (21, 1) (21,)
2 ---- 2.0 ---- (59, 1) (59,)
3 ---- 3.0 ---- (44, 1) (44,)
4 ---- 4.0 ---- (46, 1) (46,)
5 ---- 5.0 ---- (62, 1) (62,)
6 ---- 6.0 ---- (4, 1) (4,)
7 ---- 7.0 ---- (16, 1) (16,)
8 ---- 8.0 ---- (58, 1) (58,)
9 ---- 9.0 ---- (10, 1) (10,)
10 ---- 10.0 ---- (2, 1) (2,)


### Process and store the stage-1 dataset to stage-2 merged dataset

In [16]:
import cv2

In [17]:
from collections import defaultdict
class_dict: defaultdict = defaultdict(dict)
dataframes: List = []
all_images: List = []
desired_image_size = (64,101)

for idx, directory in enumerate(glob.glob("./common_processed_data/*")):
    resized_images=[]
    class_dict[idx] = os.path.basename(directory)
    df = pd.read_csv(glob.glob(path.join(directory,"*.csv"))[0])
    df["_target"]=idx
    print(df.shape)
    # open image folder and resize all images to same size that corrospond to each row in df above.
    images = np.load(glob.glob(path.join(directory,"*.npy"))[0],allow_pickle=True)
    for im in images:
        (h, w) = im.shape[:2]
        if w > h:
            # calculate the center of the image
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            im = cv2.resize(im, desired_image_size)
        else:
            im = cv2.resize(im, desired_image_size)
        resized_images.append(im)
        
    print(len(resized_images))
    # there is one dataframe for each species
    dataframes.append(df)
    # one set of images for each species
    len_res = len(resized_images)
    resized_images=np.array(resized_images)
    all_images.append(resized_images)
    if df.shape[0]==len_res:
        print('unique-->',np.unique(df["_target"]))
        print('resized_shape-->',resized_images.shape)
        print('*'*20,os.path.basename(directory),'idx-->',idx,'*'*20)
    else:
        print('failed',idx)

(2, 2)
2
unique--> [0]
resized_shape--> (2, 101, 64, 3)
******************** Acantharian idx--> 0 ********************
(21, 2)
21
unique--> [1]
resized_shape--> (21, 101, 64, 3)
******************** Alexandrium idx--> 1 ********************
(59, 2)
59
unique--> [2]
resized_shape--> (59, 101, 64, 3)
******************** Acanthoica_quattrospina idx--> 2 ********************
(44, 2)
44
unique--> [3]
resized_shape--> (44, 101, 64, 3)
******************** Euterpina_copepod idx--> 3 ********************
(46, 2)
46
unique--> [4]
resized_shape--> (46, 101, 64, 3)
******************** Empty_lorica idx--> 4 ********************
(62, 2)
62
unique--> [5]
resized_shape--> (62, 101, 64, 3)
******************** Amphorelloides_tropidoneis idx--> 5 ********************
(4, 2)
4
unique--> [6]
resized_shape--> (4, 101, 64, 3)
******************** Amphidinium_crassipes idx--> 6 ********************
(16, 2)
16
unique--> [7]
resized_shape--> (16, 101, 64, 3)
******************** Echinoderm_larvae idx--> 7 *

In [18]:
# join all species dataframes into one "master" dataframe
path_to_merged = "./common_merged_data"
os.makedirs(path_to_merged,exist_ok=True)
df: pd.DataFrame = pd.concat(dataframes, sort=True)
# store information for the entire dataset
df.to_csv(path.join(path_to_merged, 'merged_df.csv'))
# free up memory
del dataframes
# similarly stack all images into one numpy matrix
all_images: np.array = np.vstack(all_images)
np.save(path.join(path_to_merged, 'merged_images.npy'),all_images)
np.save(path.join(path_to_merged, "class_dict.npy"),class_dict)

In [19]:
df=pd.read_csv("./common_merged_data/merged_df.csv")

In [20]:
all_images=np.load("./common_merged_data/merged_images.npy")

In [21]:
class_dict=np.load("./common_merged_data/class_dict.npy",allow_pickle=True)

In [22]:
df.shape,all_images.shape

((324, 3), (324, 101, 64, 3))

In [23]:
class_dict.tolist().__len__()

11

In [24]:
df._target.unique().__len__()

11

In [25]:
df.isna().sum(axis=0)

Unnamed: 0.1    0
Unnamed: 0      0
_target         0
dtype: int64

### Process and store the stage-2 dataset to stage-3 final split (training, validation and test) dataset

In [26]:
from data_preprocessing import (drop_columns, prepare_training_data, process_attributes)

# STAGE TWO - DATA PREPROCESSING
# This includes feature engineering, standardization and data splitting
# deal with missing values / feature engineering
# df: pd.DataFrame = df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
# drop features that are no longer needed for training

print(f'total columns after modifying df -- {len(df.columns)}')

min_samples=10

''' a custom standard scaler is called from utils inside prepare training data and 
    rescaler is stored at path below using joblib.dumb'''
'''dump(rescaler, './processed_data/std_scaler.bin', compress=True)'''

os.makedirs('./common_split_data',exist_ok=True)
path_to_store_std_scaler = './common_split_data/std_scaler.bin'

trainAttrX, valAttrX, testAttrX, trainImagesX, \
    valImagesX, testImagesX, y_train, y_val, y_test = \
    prepare_training_data(df, all_images, min_samples, path_to_store_std_scaler)

total columns after modifying df -- 3


In [27]:
trainAttrX.shape, valAttrX.shape, testAttrX.shape, trainImagesX.shape, \
    valImagesX.shape, testImagesX.shape, y_train.shape, y_val.shape, y_test.shape

((227, 2),
 (45, 2),
 (44, 2),
 (227, 101, 64, 3),
 (45, 101, 64, 3),
 (44, 101, 64, 3),
 (227, 11),
 (45, 11),
 (44, 11))

In [28]:
# save the class dictionary, for later use,
# each int key maps to string species value
np.save(path.join('./common_split_data/', 'class_dict.npy'),
        class_dict,allow_pickle=True)

processed_training_data_path=path.join('./common_split_data', "plankton_data_101x64_final.pkl")
# finally save the training data for future use
with open(processed_training_data_path, "wb") as f:
    pickle.dump((trainAttrX, valAttrX, testAttrX, trainImagesX,
                 valImagesX, testImagesX, y_train, y_val, y_test),
                f, protocol=4)